# 1 Data Preparation and Loading

In [1]:
# Go to project root folder
import os
os.chdir("../")
%pwd

'/workspaces/car_vs_ambulance_classification_ensemble_cnn'

In [2]:
# automatically reload all modules before executing
%load_ext autoreload
%autoreload 2 

In [3]:
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
tf.config.list_physical_devices('GPU'), tf.__version__

2024-10-13 09:27:38.627760: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-13 09:27:38.638971: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-13 09:27:38.643039: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-13 09:27:38.652449: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-13 09:27:40.109189: E external/local_xla/xla/

([], '2.17.0')

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import utils
from src.utils.image_data_loader import ImageDataLoader

In [5]:
utils.utils.gpu_clean_up()

In [6]:
PARAMS_YAML = 'params.yaml'
params_config = utils.utils.read_yaml(yaml_path=PARAMS_YAML)
params_config

2024-10-13 09:27:40,740 — utils.utils — INFO — yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully


ConfigBox({'base': {'random_seed': 42}, 'data_split': {'test_size': 0.2}, 'train': {'image_size': 224, 'batch_size': 16, 'loss': 'binary_crossentropy', 'if_fine_tune': True, 'fine_tune_args': {'models': ['lenet', 'mini_vgg_net', 'shallownet'], 'lr': {'max': 0.01, 'min': 0.0001}, 'epochs': 2, 'optimizers': ['adam', 'sgd']}, 'lenet': {'base_lr': 0.01, 'epochs': 2}, 'mini_vgg_net': {'base_lr': 0.01, 'epochs': 10}, 'shallownet': {'base_lr': 0.01, 'epochs': 10}}, 'evaluate': {'metrics': ['accuracy', 'precision', 'recall'], 'reports_dir': 'src/reports', 'results_dir': 'src/reports/results', 'confusion_matrix_image': 'confusion_matrix.png'}, 'paths': {'dataset_url': 'https://www.kaggle.com/datasets/amansingh0000000/image-classification-and-yolo-dataset/data', 'raw_dataset_dir': 'dataset/raw', 'dataset_path': 'dataset/image-classification-and-yolo-dataset/Ambulance_Car', 'preprocess_dataset': 'dataset/preprocess', 'train_dir': 'dataset/raw/image-classification-and-yolo-dataset/Ambulance_Car/Tr

In [7]:
tf.random.set_seed(params_config.base.random_seed)

In [8]:
image_loader = ImageDataLoader(params_yaml=PARAMS_YAML)
train_ds, val_ds = image_loader.load_image_dataset()

2024-10-13 09:27:40,933 — utils.utils — INFO — yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully
Found 2227 files belonging to 2 classes.


Found 557 files belonging to 2 classes.
2024-10-13 09:27:41,038 — src.utils.image_data_loader — INFO — load_image_dataset are prefetched
2024-10-13 09:27:41,038 — src.utils.image_data_loader — INFO — load_image_dataset completed


# 2 Model Training

In [9]:
from models import lenet, mini_vgg_net, shallownet
from tensorflow import keras

### LeNet Train

In [10]:
lenet_model = lenet.LeNet(optimizer_name='adam',
                          image_size=params_config.train.image_size,
                          loss=params_config.train.loss,
                          batch_size=params_config.train.batch_size,
                          epochs=params_config.train.lenet.epochs,
                          learning_rate=params_config.train.lenet.base_lr,
                          metrics=params_config.evaluate.metrics)
lenet_model.build()
lenet_model.get_summary()

2024-10-13 09:27:41,474 — models.lenet — INFO — LeNet model built successfully.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 20)   │         1,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 20)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 112, 112, 20)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 50)   │        25,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 50)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 56, 56, 50)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 156800)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 156800)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 500)            │    78,400,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 500)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           501 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 78,427,571 (299.18 MB)

 Trainable params: 78,427,571 (299.18 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
lenet_model.model_compile()

2024-10-13 09:27:41,513 — models.lenet — INFO — Selecting optimizer adam


In [12]:
# train the network
history = lenet_model.train(train_data=train_ds,
                            val_data=val_ds)

2024-10-13 09:27:41,549 — models.lenet — INFO — Selecting optimizer adam
Epoch 1/2


 48/140 ━━━━━━━━━━━━━━━━━━━━ 1:21 884ms/step - accuracy: 0.6862 - loss: 1936.2446 - precision: 0.8583 - recall: 0.7509

KeyboardInterrupt: 

In [40]:
for element in val_ds.as_numpy_iterator():
    prd = lenet_model.evaluate(element)
    print(prd)
    break

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
2024-09-28 15:35:38,359 — models.lenet — INFO — Evaluation completed
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]


### MiniVGGNet Train

In [13]:
mini_vgg_net_model = mini_vgg_net.MiniVGGNet(optimizer_name='adam',
                                             image_size=params_config.train.image_size,
                                             loss=params_config.train.loss,
                                             batch_size=params_config.train.batch_size,
                                             epochs=params_config.train.mini_vgg_net.epochs,
                                             learning_rate=params_config.train.mini_vgg_net.base_lr,
                                             metrics=params_config.evaluate.metrics)
mini_vgg_net_model.build()
mini_vgg_net_model.get_summary()

2024-10-13 09:28:30,079 — models.mini_vgg_net — INFO — MiniVGGNet model built successfully.


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 224, 224, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 112, 112, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 200704)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │   102,760,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 102,829,857 (392.26 MB)

 Trainable params: 102,828,449 (392.26 MB)

 Non-trainable params: 1,408 (5.50 KB)

In [14]:
# train the network
history = mini_vgg_net_model.train(train_data=train_ds,
                                   val_data=val_ds)

2024-10-13 09:53:50,301 — models.mini_vgg_net — INFO — Selecting optimizer adam
Epoch 1/2
140/140 ━━━━━━━━━━━━━━━━━━━━ 324s 2s/step - accuracy: 0.7305 - loss: 0.8232 - precision: 0.8559 - recall: 0.8182 - val_accuracy: 0.6284 - val_loss: 0.9830 - val_precision: 0.9075 - val_recall: 0.6333
Epoch 2/2
140/140 ━━━━━━━━━━━━━━━━━━━━ 412s 3s/step - accuracy: 0.8265 - loss: 0.4608 - precision: 0.8789 - recall: 0.9217 - val_accuracy: 0.8384 - val_loss: 0.9395 - val_precision: 0.8980 - val_recall: 0.9167
2024-10-13 10:06:06,289 — models.mini_vgg_net — INFO — Training completed.


### ShallowNet Train

In [43]:
shallownet_model = shallownet.ShallowNet(params_yaml=PARAMS_YAML)
shallownet_model.build()
shallownet_model.get_summary()

2024-09-28 15:40:46,991 — utils.utils — INFO — yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully
2024-09-28 15:40:47,001 — models.shallownet — INFO — ShallowNet model built successfully.


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_19 (Conv2D)              │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 1605632)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │     1,605,633 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,606,529 (6.13 MB)

 Trainable params: 1,606,529 (6.13 MB)

 Non-trainable params: 0 (0.00 B)